## Load SO, POD, NAV

### IN: open purchase orders & Sales Date return platform
### OUT: open sales orders
### Balance: Inventory

In [126]:
import pandas as pd
import csv
import string
import os
from pathlib import Path
from datetime import datetime
import pandas as pd
from sqlalchemy import create_engine

# Build Supabase engine
DATABASE_DSN = (
    "postgresql://postgres.avcznjglmqhmzqtsrlfg:Czheyuan0227@"
    "aws-0-us-east-2.pooler.supabase.com:6543/postgres?sslmode=require"
)
engine = create_engine(DATABASE_DSN, pool_pre_ping=True)

#SO 
#Backordered是指目前SO尚未出貨的數量，如果用QTY的話會包含已經partial出貨的
# SO = pd.read_csv("Data/open sales orders.csv", encoding="utf-8", encoding_errors="ignore")
SO = pd.read_sql_table("open_sales_orders", con=engine, schema="public") # Pull from Supabase
SO = SO.drop(columns=["Qty"], axis =1)
SO.rename(columns={"Date":"Order Date","Num":"QB Num","Backordered":"Qty(-)"},inplace=True)
SO = SO.drop(SO.columns[[0]], axis =1)
SO = SO.drop(columns=['Type','Due Date','Terms','Amount','Deliv Date','Open Balance',"Invoiced","Rep"], axis =1)
SO = SO.dropna(axis=0, how='all',subset=None, inplace=False)
SO = SO.dropna(thresh=6)
SO['Item']= SO['Item'].str.split(':',expand=True)[1]
SO['Item']= SO['Item'].str.replace("*","")
SO['Qty(+)']="0"
SO['Remark']=""
SO['Order Date']= pd.to_datetime(SO['Order Date'])
SO['Order Date'] = SO['Order Date'].dt.strftime('%Y/%m/%d')
SO['Ship Date']= pd.to_datetime(SO['Ship Date'])
SO['Ship Date'] = SO['Ship Date'].dt.strftime('%Y/%m/%d')
columns = ['Order Date','Ship Date', 'QB Num',"P. O. #","Name",'Qty(+)','Qty(-)', 'Item','Inventory Site','Remark']



#POD (Could be replaced by "C:\Users\Admin\OneDrive - neousys-tech\NTA OneDrive's files - 00 NTA POD list_item purchased record\POD list, inventory management_V.xlsx")
pod = pd.read_csv("Data/open purchase orders.csv", encoding="ISO-8859-1")
pod = pod.drop(columns=['Name','Amount','Open Balance',"Rcv'd","Qty"], axis =1)
pod.rename(columns={"Date":"Order Date","Num":"QB Num","Source Name":"Name","Backordered":"Qty(+)"},inplace=True)
pod = pod.drop(pod.columns[[0]], axis =1)
pod = pod.dropna(axis=0, how='all',subset=None, inplace=False)
pod = pod.dropna(thresh=5)
pod['Memo'] = pod['Memo'].str.split(' ',expand=True)[0]
pod['QB Num'] = pod['QB Num'].str.split('(',expand=True)[0]
print(pod['Memo'].str.split('*',expand=True)[0])
pod['Memo'] = pod['Memo'].str.replace("*","")
pod.rename(columns={"Memo":"Item"},inplace=True)
pod['Order Date']= pd.to_datetime(pod['Order Date'])
pod['Deliv Date']= pd.to_datetime(pod['Deliv Date'])
pod['Order Date'] = pod['Order Date'].dt.strftime('%Y/%m/%d')
pod['Deliv Date'] = pod['Deliv Date'].dt.strftime('%Y/%m/%d')
pod.to_csv('open purchase2.csv',index=False)

#NAV
NAV = pd.read_csv("Data/Sales Date return platform.csv",usecols=['Document No.',"Customer PO No.","Customer Ordering Model","OP Estimated Shipping Date","Quantity","No.","Customer Ordering Desc."],encoding='utf-8')
NAV.rename(columns={"Customer PO No.":"QB Num","Customer Ordering Model":"Item",'Document No.':"Remark","OP Estimated Shipping Date":"Ship Date","Quantity":"Qty(+)"},inplace=True)
NAV['QB Num'] = NAV['QB Num'].str.split('(',expand=True)[0]
NAV.to_csv('NAV1.csv',index=False)
# with open('NAV1.csv', 'r',encoding='utf-8') as file:
#     csv_reader = csv.reader(file)

# Convert DataFrame rows into list of lists
data_list = NAV.values.tolist()
n = len(data_list)
s50 = []
for i in range(n):
    if str(data_list[i][2]).startswith("S"):  # check Item column
        s50.append(data_list[i])

# s50 = pd.DataFrame(s50, columns=NAV.columns)

3                    AccsyBx-6AntiVG-POC-551VTC
4                    AccsyBx-6AntiVG-POC-551VTC
5                    AccsyBx-6AntiVG-POC-551VTC
6                    AccsyBx-6AntiVG-POC-551VTC
9       AccsyBx-Cardholder-10108GC-5080_70_70Ti
                         ...                   
1160                                           
1161                              Certification
1162                                   Engineer
1163                                   Engineer
1167                                     Tariff
Name: 0, Length: 574, dtype: object


## Load Inventory Status

In [98]:
import pandas as pd
from sqlalchemy import create_engine

# 1. Build Supabase engine
DATABASE_DSN = (
    "postgresql://postgres.avcznjglmqhmzqtsrlfg:Czheyuan0227@"
    "aws-0-us-east-2.pooler.supabase.com:6543/postgres?sslmode=require"
)
engine = create_engine(DATABASE_DSN, pool_pre_ping=True)

# 2. Pull Inventory Status
INV = pd.read_sql_table("inventory_status", con=engine, schema="public")
INV

,Part_Number,Reorder Pt (Min),Max,On Hand,On Sales Order,Stock_Available,Order,On PO,Reorder Qty,Next Deliv,Sales/Week
0,WH01S-NTA,NaN,NaN,0.0,0.0,0.0,None,0.0,NaN,None,NaN
1,Inventory,NaN,NaN,0.0,0.0,0.0,None,0.0,NaN,None,NaN
2,Accessory,NaN,NaN,0.0,0.0,0.0,None,0.0,NaN,None,NaN
3,303-13200005-000,NaN,NaN,0.0,0.0,0.0,None,0.0,0.0,None,0.0
4,AccsyBx-10208GC-Chassis-NTA(EA),NaN,NaN,1.0,0.0,1.0,None,0.0,0.0,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1657,Win11IoT24-Value,NaN,NaN,16.0,11.0,5.0,None,0.0,0.0,None,0.6
1658,Win7Pro-Emb,NaN,NaN,0.0,0.0,0.0,None,0.0,0.0,None,0.6
1659,Total Windows OS,NaN,NaN,227.0,294.0,-67.0,None,112.0,NaN,None,22.7
1660,Total Inventory,NaN,NaN,7386.0,7807.0,-421.0,None,5445.0,NaN,None,689.2


In [196]:
result_lists = []

# 1) Split Description into base model and included components, then expand
for original_list in s50:
    desc = str(original_list[-1]).strip()

    if ', including' in desc:
        head, tail = desc.split(', including', 1)
        product_code = head.split(',')[0].strip()

        # Components list
        components = [c.strip() for c in tail.split(',') if c.strip()]

        # (A) Add a row for each included component
        for comp in components:
            new_list = original_list.copy()
            new_list[-1] = comp                         # put component name in the Description slot
            new_list.append('Pre-installed Component')        # <-- NEW FLAG COLUMN
            result_lists.append(new_list)

        # (B) Add a row for the base product itself
        base_row = original_list.copy()
        base_row[-1] = product_code                      # put base model in Description slot
        base_row.append('Base Unit')                     # <-- NEW FLAG COLUMN
        result_lists.append(base_row)

    else:
        # No “including” → keep as-is but mark as standalone
        row = original_list.copy()
        row.append('Standalone (No includes)')           # <-- NEW FLAG COLUMN
        result_lists.append(row)

# This keeps your old mapping intact (indices before the appended flag remain unchanged)
for i in range(len(result_lists)):
    result_lists[i][3] = result_lists[i][6]

# 2) Normalize and expand quantities
transformed_lists = []
for result_list in result_lists:
    transformed_list = result_list.copy()
    # assuming name-in-col-3, qty-in-col-4 (as in your code)
    transformed_list[3] = transformed_list[3].replace(" ", "")
    if len(transformed_list[3]) >= 2 and transformed_list[3][0].isdigit() and transformed_list[3][1] == 'x':
        qty = int(transformed_list[3].split('x', 1)[0])
        name = transformed_list[3].split('x', 1)[1]
        transformed_list[3] = name
        transformed_list[4] = str(qty * float(transformed_list[4]))
    transformed_lists.append(transformed_list)

## 3.Normalize and expand quantities
# 迭代原始 list
for result_list in result_lists:
    # 複製原始 list進行修改
    transformed_list = result_list.copy()
    for i in range(len(transformed_list)):
        transformed_list[3]=transformed_list[3].replace(" ","")
        if 'x' == transformed_list[3][1] and transformed_list[3][0].isdigit() == True:
            # 提取數量
            quantity = int(transformed_list[3].split('x')[0])
            # 提取原始名稱
            name = transformed_list[3].split('x')[-1]
            # 替換為修改後的項目
            transformed_list[3] = name
            # 修改數量和版本號
            transformed_list[4] = str(quantity * float(transformed_list[4]))

    # 將修改後的列表添加到轉換後的列表中
    transformed_lists.append(transformed_list)

NAV = pd.DataFrame(transformed_lists)

# If you know your column names, set them here:
NAV.columns = ['Remark','QB Num','No.','Item','Qty(+)','Ship Date', 'Need to Drop', 'Install Type']

#NAV加上倉別跟日期
# NAV = pd.read_csv("NAV1.csv",usecols=['Remark','QB Num','Item','Qty(+)','Ship Date'],encoding='utf-8')
# a = pd.read_csv('open purchase2.csv',usecols=['QB Num',"Order Date","Inventory Site","P. O. #","Name"])
pod.drop_duplicates(inplace=True)
pod['Qty(-)']="0"

Final=pd.merge(left=NAV,right=pod,on=["QB Num", "Item"],how="left") # add "Item"
columns = ['Order Date','Ship Date', 'QB Num',"P. O. #","Name",'Qty(-)','Qty(+)', 'Item','Inventory Site','Remark']

In [197]:
NAV.head(5)

,Remark,QB Num,No.,Item,Qty(+),Ship Date,Need to Drop,Install Type
0,SO25050034,POD-250647,S51-SL1708FF-003,i7-9700TE,10,10/1/2025,i7-9700TE,Pre-installed Component
1,SO25050034,POD-250647,S51-SL1708FF-003,DDR4-16GB-32-SMandM.280-SSD-1TB-PCIe44-TLC5-PN,10,10/1/2025,DDR4-16GB-32-SM and M.280-SSD-1TB-PCIe44-TLC5-PN,Pre-installed Component
2,SO25050034,POD-250647,S51-SL1708FF-003,SEMIL-1708-FF,10,10/1/2025,SEMIL-1708-FF,Base Unit
3,SO25050035,POD-250648,S51-SL1708FF-003,i7-9700TE,10,11/19/2025,i7-9700TE,Pre-installed Component
4,SO25050035,POD-250648,S51-SL1708FF-003,DDR4-16GB-32-SMandM.280-SSD-1TB-PCIe44-TLC5-PN,10,11/19/2025,DDR4-16GB-32-SM and M.280-SSD-1TB-PCIe44-TLC5-PN,Pre-installed Component


In [189]:
pod.head(5)

,Order Date,QB Num,Name,Item,Deliv Date,Qty(+),Qty(-)
3,2025/07/24,POD-251045,Neousys Technology Incorp.,AccsyBx-6AntiVG-POC-551VTC,2025/09/26,36.0,0
4,2025/07/24,POD-251046,Neousys Technology Incorp.,AccsyBx-6AntiVG-POC-551VTC,2025/10/03,34.0,0
5,2025/07/24,POD-251047,Neousys Technology Incorp.,AccsyBx-6AntiVG-POC-551VTC,2025/11/21,24.0,0
6,2025/07/24,POD-251048,Neousys Technology Incorp.,AccsyBx-6AntiVG-POC-551VTC,2025/12/19,29.0,0
9,2025/09/08,POD-251277,Neousys Technology Incorp.,AccsyBx-Cardholder-10108GC-5080_70_70Ti,2025/09/24,1.0,0


In [145]:
result_lists = []
for original_list in s50:
    # 分割字串
    product_info = str(original_list[-1]).split(', including ')
    product_info[0] = product_info[0].split(',')[0]
    # print(product_info[0])
    product_code = product_info[0]
    #print(product_info[1])
    if len(product_info) > 1:
        components = product_info[1].split(', ')
    else:
        None

    
    # 建立各組件的新 list
    for component in components:
        new_list = original_list.copy()
        new_list[-1] = component
        new_list[-1] = new_list[-1].strip(" ")
        result_lists.append(new_list)

    # 將產品代碼單獨加入新的 list
    new_list_with_product_code = original_list.copy()
    new_list_with_product_code[-1] = product_code
    result_lists.append(new_list_with_product_code)
for i in range(0,len(result_lists)):
    result_lists[i][3] = result_lists[i][6]

# 轉換後的 list
transformed_lists = []

# 迭代原始 list
for result_list in result_lists:
    # 複製原始 list進行修改
    transformed_list = result_list.copy()
    for i in range(len(transformed_list)):
        transformed_list[3]=transformed_list[3].replace(" ","")
        if 'x' == transformed_list[3][1] and transformed_list[3][0].isdigit() == True:
            # 提取數量
            quantity = int(transformed_list[3].split('x')[0])
            # 提取原始名稱
            name = transformed_list[3].split('x')[-1]
            # 替換為修改後的項目
            transformed_list[3] = name
            # 修改數量和版本號
            transformed_list[4] = str(quantity * float(transformed_list[4]))

    # 將修改後的列表添加到轉換後的列表中
    transformed_lists.append(transformed_list)

   
csvfile = open('NAV1.csv', 'a+',encoding='utf-8', newline ="")
with csvfile:
    write = csv.writer(csvfile)
    for i in range(0,len(transformed_lists)):
        a = transformed_lists[i]
        write.writerow(a)

csvfile.close()

#NAV加上倉別跟日期
NAV = pd.read_csv("NAV1.csv",usecols=['Remark','QB Num','Item','Qty(+)','Ship Date'],encoding='utf-8')
# a = pd.read_csv('open purchase2.csv',usecols=['QB Num',"Order Date","Inventory Site","P. O. #","Name"])
pod.drop_duplicates(inplace=True)
pod['Qty(-)']="0"

Final=pd.merge(left=NAV,right=pod,on=["QB Num", "Item"],how="left")
columns = ['Order Date','Ship Date', 'QB Num',"P. O. #","Name",'Qty(-)','Qty(+)', 'Item','Inventory Site','Remark']


SONAV = pd.concat([SO,Final])
SONAV = SONAV.sort_values(by=["Inventory Site","Item","Ship Date"], ascending=False)
SONAV

,Order Date,Ship Date,QB Num,P. O. #,Name,Qty(-),Item,Inventory Site,Qty(+),Remark,Qty(+)_x,Deliv Date,Qty(+)_y
632,2025/08/15,2025/12/31,SO-20251167,PO5382,"Dexterity, Inc.",1.0,mPCIe-CAN-IPEH-4047,WH601C-NT,0,,NaN,NaN,NaN
630,2025/08/14,2025/10/10,SO-20251162,454361,Navico Group Americas LLC,1.0,mPCIe-CAN-IPEH-4047,WH601C-NT,0,,NaN,NaN,NaN
629,2025/08/01,2025/10/01,SO-20251095,PO5343,"Dexterity, Inc.",1.0,mPCIe-CAN-IPEH-4047,WH601C-NT,0,,NaN,NaN,NaN
1784,2025/08/15,2025/12/31,SO-20251167,PO5382,"Dexterity, Inc.",4.0,SSD-4TB-TLC-SS,WH601C-NT,0,,NaN,NaN,NaN
1783,2025/08/01,2025/10/01,SO-20251095,PO5343,"Dexterity, Inc.",4.0,SSD-4TB-TLC-SS,WH601C-NT,0,,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1900,2025/04/24,2025/07/23,SO-20250601,41477111,Inficon Inc.,23.0,None,None,0,,NaN,NaN,NaN
2043,2025/04/24,2025/07/23,SO-20250600,41477111,Inficon Inc.,1.0,None,None,0,,NaN,NaN,NaN
2044,2025/04/24,2025/07/23,SO-20250601,41477111,Inficon Inc.,1.0,None,None,0,,NaN,NaN,NaN
1910,2025/06/06,2025/07/09,SO-20250800,AI-001279,"Applied Intuition, Inc.",2.0,None,None,0,,NaN,NaN,NaN


In [161]:
SONAV[SONAV["Item"]=="i7-14700"]

,Order Date,Ship Date,QB Num,P. O. #,Name,Qty(-),Item,Inventory Site,Qty(+),Remark,Qty(+)_x,Deliv Date,Qty(+)_y
440,2025/09/04,2025/12/31,SO-20251259,405,Candela Technologies Inc.,1.0,i7-14700,WH01S-NTA,0,,NaN,NaN,NaN
441,2025/09/08,2025/12/31,SO-20251269,8014.07-01,"ASI Technology, Inc.",1.0,i7-14700,WH01S-NTA,0,,NaN,NaN,NaN
438,2025/08/28,2025/10/21,SO-20251229,CYNGN-1275,CYNGN INC.,4.0,i7-14700,WH01S-NTA,0,,NaN,NaN,NaN
439,2025/08/28,2025/09/24,SO-20251237,PO24530,"E-Technologies Group, LLC",1.0,i7-14700,WH01S-NTA,0,,NaN,NaN,NaN
442,2025/09/09,2025/09/24,SO-20251276,PO24717,"E-Technologies Group, LLC",1.0,i7-14700,WH01S-NTA,0,,NaN,NaN,NaN
437,2025/08/28,2025/09/19,SO-20251230,250828A,Ball Horticultural Company,1.0,i7-14700,WH01S-NTA,0,,NaN,NaN,NaN
576,2025/09/12,NaN,POD-251308,NaN,Neousys Technology Incorp.,0,i7-14700,nan,NaN,SO25090277,1.0,2025/10/01,1.0
819,2025/09/12,NaN,POD-251308,NaN,Neousys Technology Incorp.,0,i7-14700,nan,NaN,SO25090277,1.0,2025/10/01,1.0


In [178]:
pod[pod["Item"]=="TB-10"]

,Order Date,QB Num,Name,Item,Deliv Date,Qty(+),Qty(-)
176,2025/07/25,POD-251061,Neousys Technology Incorp.,TB-10,2025/08/13,5.0,0
177,2025/07/25,POD-251062,Neousys Technology Incorp.,TB-10,2025/12/24,5.0,0


In [192]:
NAV[NAV["Item"]=="TB-10"]

,Remark,QB Num,No.,Item,Qty(+),Ship Date,Need to Drop,Install Type


In [ ]:
NAV[NAV["QB Num"]=="POD-251061"]

,Remark,QB Num,No.,Item,Qty(+),Ship Date,Need to Drop,Install Type
37,SO25070495,POD-251061,S51-UC0715CG-001,DDR5-16GB-56-SM,5,9/24/2025,DDR5-16GB-56-SM,Pre-installed Component
38,SO25070495,POD-251061,S51-UC0715CG-001,M.280-SSD-256GB-PCIe44-TLC5-PN,5,9/24/2025,M.280-SSD-256GB-PCIe44-TLC5-PN,Pre-installed Component
39,SO25070495,POD-251061,S51-UC0715CG-001,mPCIe-Ado-AX92905,5,9/24/2025,mPCIe-Ado-AX92905,Pre-installed Component
40,SO25070495,POD-251061,S51-UC0715CG-001,POC-715-CGN1,5,9/24/2025,POC-715-CGN1,Base Unit
41,SO25070495,POD-251061,S51-UC0715C1-001,DDR5-16GB-56-SM,10,9/24/2025,DDR5-16GB-56-SM,Pre-installed Component
42,SO25070495,POD-251061,S51-UC0715C1-001,M.280-SSD-256GB-PCIe44-TLC5-PN,10,9/24/2025,M.280-SSD-256GB-PCIe44-TLC5-PN,Pre-installed Component
43,SO25070495,POD-251061,S51-UC0715C1-001,mPCIe-Ado-AX92905,10,9/24/2025,mPCIe-Ado-AX92905,Pre-installed Component
44,SO25070495,POD-251061,S51-UC0715C1-001,POC-715-CGN2,10,9/24/2025,POC-715-CGN2,Base Unit
262,SO25070495,POD-251061,S51-UC0715CG-001,DDR5-16GB-56-SM,5,9/24/2025,DDR5-16GB-56-SM,Pre-installed Component
263,SO25070495,POD-251061,S51-UC0715CG-001,M.280-SSD-256GB-PCIe44-TLC5-PN,5,9/24/2025,M.280-SSD-256GB-PCIe44-TLC5-PN,Pre-installed Component


## Qty by Item by date

In [133]:
import pandas as pd
from datetime import datetime, timedelta
from datetime import date

def build_timephased_qty(
    start_date=None,
    end_date=None,
    freq="D",
    initial_onhand_df=None,   # pass INV here
    site_filter="WH01S-NTA",  # focus on this site
):
    df = SONAV

    # ---- Filter Site ----
    df["Inventory Site"] = df["Inventory Site"].astype(str).str.strip()
    if site_filter is not None:
        df = df[df["Inventory Site"] == site_filter].copy()

    # clean numeric & date
    df["Qty(+)"] = pd.to_numeric(df["Qty(+)"], errors="coerce").fillna(0)
    df["Qty(-)"] = pd.to_numeric(df["Qty(-)"], errors="coerce").fillna(0)
    df["Ship Date"] = pd.to_datetime(df["Ship Date"], errors="coerce")
    df = df.dropna(subset=["Ship Date"])
    df["Item"] = df["Item"].astype(str).str.strip()

    # net movement per date
    df["qty_change"] = df["Qty(+)"] - df["Qty(-)"]
    events = (df.groupby(["Inventory Site","Item","Ship Date"], as_index=False)["qty_change"]
                .sum()
                .sort_values(["Inventory Site","Item","Ship Date"]))

    # horizon
    if start_date is None:
        start = pd.Timestamp("2025-01-01")   # or use today().date()
    else:
        start = pd.to_datetime(start_date).normalize()

    if end_date is None:
        end = start + timedelta(days=30)
    else:
        end = pd.to_datetime(end_date).normalize()

    date_index = pd.date_range(start, end, freq=freq)

    # all (site,item) pairs from events
    pairs = events[["Inventory Site","Item"]].drop_duplicates()

    # build continuous timeline
    all_rows = []
    for (site, item), grp in events.groupby(["Inventory Site","Item"]):
        s = grp.set_index("Ship Date")["qty_change"].reindex(date_index, fill_value=0.0)
        out = pd.DataFrame({
            "Ship Date": date_index,
            "Inventory Site": site,
            "Item": item,
            "Net Movement": s.values
        })
        all_rows.append(out)

    timeline = pd.concat(all_rows, ignore_index=True) if all_rows else pd.DataFrame(
        columns=["Ship Date","Inventory Site","Item","Net Movement"]
    )

    # ----- add initial on-hand (INV) -----
    # Case: INV has NO site -> broadcast to the chosen site
    if initial_onhand_df is not None and not initial_onhand_df.empty:
        onhand = initial_onhand_df.copy()
        # normalize columns from INV
        if "Part_Number" in onhand.columns:
            onhand = onhand.rename(columns={"Part_Number":"Item"})
        onhand["Item"] = onhand["Item"].astype(str).str.strip()
        onhand["On Hand"] = pd.to_numeric(onhand["On Hand"], errors="coerce").fillna(0)

        if "Inventory Site" not in onhand.columns:
            onhand["Inventory Site"] = site_filter  # broadcast INV to this site

        onhand = onhand[["Inventory Site","Item","On Hand"]]
        # limit to items present in timeline to avoid bloat
        if not timeline.empty:
            onhand = onhand.merge(pairs, on=["Inventory Site","Item"], how="inner")

        timeline = timeline.merge(onhand, on=["Inventory Site","Item"], how="left")
        timeline["On Hand"] = timeline["On Hand"].fillna(0.0)
    else:
        timeline["On Hand"] = 0.0

    # running balance
    if not timeline.empty:
        timeline.sort_values(["Inventory Site","Item","Ship Date"], inplace=True)
        cum_net = timeline.groupby(["Inventory Site","Item"])["Net Movement"].cumsum()
        init_onhand = timeline.groupby(["Inventory Site","Item"])["On Hand"].transform("first")
        timeline["Projected Qty"] = init_onhand + cum_net
    else:
        timeline["Projected Qty"] = pd.Series(dtype=float)

    return events, timeline



In [159]:
# Prepare INV (no site column in your screenshot)
initial_onhand_df = INV[["Part_Number","On Hand"]].copy()

events, timeline = build_timephased_qty(
    start_date=date.today(),
    end_date=None,
    freq="D",
    initial_onhand_df=initial_onhand_df,
    site_filter="WH01S-NTA",
)

part_view = timeline[
    (timeline["Item"] == "TB-10") & (timeline["Inventory Site"] == "WH01S-NTA")
]
print(part_view[["Ship Date","On Hand","Net Movement","Projected Qty"]].head(25))



      Ship Date  On Hand  Net Movement  Projected Qty
6944 2025-09-18      9.0           0.0            9.0
6945 2025-09-19      9.0           0.0            9.0
6946 2025-09-20      9.0           0.0            9.0
6947 2025-09-21      9.0           0.0            9.0
6948 2025-09-22      9.0           0.0            9.0
6949 2025-09-23      9.0           0.0            9.0
6950 2025-09-24      9.0           0.0            9.0
6951 2025-09-25      9.0           0.0            9.0
6952 2025-09-26      9.0           0.0            9.0
6953 2025-09-27      9.0           0.0            9.0
6954 2025-09-28      9.0           0.0            9.0
6955 2025-09-29      9.0           0.0            9.0
6956 2025-09-30      9.0          -5.0            4.0
6957 2025-10-01      9.0           0.0            4.0
6958 2025-10-02      9.0           0.0            4.0
6959 2025-10-03      9.0           0.0            4.0
6960 2025-10-04      9.0           0.0            4.0
6961 2025-10-05      9.0    

In [111]:
timeline[timeline["Item"]=="i7-14700"]

,Ship Date,Inventory Site,Item,Net Movement,On Hand,Projected Qty
7223,2025-09-17,WH01S-NTA,i7-14700,0.0,17.0,17.0
7224,2025-09-18,WH01S-NTA,i7-14700,0.0,17.0,17.0
7225,2025-09-19,WH01S-NTA,i7-14700,0.0,17.0,17.0
7226,2025-09-20,WH01S-NTA,i7-14700,0.0,17.0,17.0
7227,2025-09-21,WH01S-NTA,i7-14700,0.0,17.0,17.0
7228,2025-09-22,WH01S-NTA,i7-14700,0.0,17.0,17.0
7229,2025-09-23,WH01S-NTA,i7-14700,0.0,17.0,17.0
7230,2025-09-24,WH01S-NTA,i7-14700,0.0,17.0,17.0
7231,2025-09-25,WH01S-NTA,i7-14700,0.0,17.0,17.0
7232,2025-09-26,WH01S-NTA,i7-14700,0.0,17.0,17.0
